# Mcdonald vs Burger king
A Financial & Operational Showdown Between Burger King and McDonald’s (2021–2023)

## About Dataset
This project compares the financial performance and business strategies of Burger King and McDonald’s between 2021 and 2023.

The workflow began with raw data for both companies, which was cleaned and structured in Excel. Once cleaned, the datasets were converted into CSV files for use in Python. Using Python (with Pandas), the data was transformed, aggregated, and prepared for visualization.

# Data: Burger King - Global Sales Performance (2021 - 2023)
**Description:** This CSV file contains aggregated annual performance metrics for Burger King from 2021 to 2023, segmented by metric type and region scope (Global/U.S.).

In [0]:
df_bk = spark.read.format("csv").option("header", "true").load("/Volumes/workspace/my_data/performance_analysis/bk.csv")
display(df_bk)

# Data: McDonalds - Global Sales Performance (2021 - 2023)
**Description:** This CSV file contains aggregated annual performance metrics for McDonalds from 2021 to 2023, segmented by metric type and region scope (Global/U.S.).

In [0]:
df_mcd = spark.read.format("csv").option("header", "true").load("/Volumes/workspace/my_data/performance_analysis/mcd.csv")
display(df_mcd)